***BARILLER Halvard - EVE Célestin - TERVER Basile*** <br>

***MVA 23/24*** <br>

## <center> Deep Learning in Practice
# <center> Visualization of CNN: Grad-CAM

* **Objective**: Convolutional Neural Networks are widely used on computer vision. It is powerful for processing grid-like data. However we hardly know how and why it works, due to the lack of decomposability into individually intuitive components. In this assignment, we use Grad-CAM, which highlights the regions of the input image that were important for the neural network prediction.


* NB: if `PIL` is not installed, try `conda install pillow`.

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import models, datasets, transforms
import matplotlib.pyplot as plt
import pickle
import urllib.request

import numpy as np
from PIL import Image

%matplotlib inline

### Download the Model
We provide you a pretrained model `ResNet-34` for `ImageNet` classification dataset.
* **ImageNet**: A large dataset of photographs with 1 000 classes.
* **ResNet-34**: A deep architecture for image classification.

In [ ]:
resnet34 = models.resnet34(weights='ResNet34_Weights.IMAGENET1K_V1')  # New PyTorch interface for loading weights!
resnet34.eval() # set the model to evaluation mode

![ResNet34](https://miro.medium.com/max/1050/1*Y-u7dH4WC-dXyn9jOG4w0w.png)


Input image must be of size (3x224x224). 

First convolution layer with maxpool. 
Then 4 ResNet blocks. 

Output of the last ResNet block is of size (512x7x7). 

Average pooling is applied to this layer to have a 1D array of 512 features fed to a linear layer that outputs 1000 values (one for each class). No softmax is present in this case. We have already the raw class score!

In [ ]:
classes = pickle.load(urllib.request.urlopen('https://gist.githubusercontent.com/yrevar/6135f1bd8dcf2e0cc683/raw/d133d61a09d7e5a3b36b8c111a8dd5c4b5d560ee/imagenet1000_clsid_to_human.pkl'))

##classes is a dictionary with the name of each class 
# print(classes)
# print(len(classes))

### Input Images
We provide you 20 images from ImageNet (download link on the webpage of the course or download directly using the following command line,).<br>
In order to use the pretrained model resnet34, the input image should be normalized using `mean = [0.485, 0.456, 0.406]`, and `std = [0.229, 0.224, 0.225]`, and be resized as `(224, 224)`.

In [ ]:
def preprocess_image(dir_path):
    normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                     std=[0.229, 0.224, 0.225])
    # Note: If the inverse normalisation is required, apply 1/x to the above object
    
    dataset = datasets.ImageFolder(dir_path, transforms.Compose([
            transforms.Resize(256), 
            transforms.CenterCrop(224), # resize the image to 224x224
            transforms.ToTensor(), # convert numpy.array to tensor
            normalize])) #normalize the tensor

    return (dataset)

In [ ]:
import os
if not os.path.exists("data"):
    os.mkdir("data")
if not os.path.exists("data/TP2_images"):
    os.mkdir("data/TP2_images")
    !cd data/TP2_images && wget "https://www.lri.fr/~gcharpia/deeppractice/2023/TP2/TP2_images.zip" && unzip TP2_images.zip

dir_path = "data/" 
dataset = preprocess_image(dir_path)

In [ ]:
# show the original image 
index = 5
input_image = Image.open(dataset.imgs[index][0]).convert('RGB')
plt.imshow(input_image)

In [ ]:
output = resnet34(dataset[index][0].view(1, 3, 224, 224))
values, indices = torch.topk(output, 3)
print("Top 3-classes:", indices[0].numpy(), [classes[x] for x in indices[0].numpy()])
print("Raw class scores:", values[0].detach().numpy())

### Grad-CAM 
* **Overview:** Given an image, and a category (‘tiger cat’) as input, we forward-propagate the image through the model to obtain the `raw class scores` before softmax. The gradients are set to zero for all classes except the desired class (tiger cat), which is set to 1. This signal is then backpropagated to the `rectified convolutional feature map` of interest, where we can compute the coarse Grad-CAM localization (blue heatmap).


* **To Do**: Define your own function Grad_CAM to achieve the visualization of the given images. For each image, choose the top-3 possible labels as the desired classes. Compare the heatmaps of the three classes, and conclude. 


* **To be submitted within 2 weeks**: this notebook, **cleaned** (i.e. without results, for file size reasons: `menu > kernel > restart and clean`), in a state ready to be executed (if one just presses 'Enter' till the end, one should obtain all the results for all images) with a few comments at the end. No additional report, just the notebook!


* **Hints**: 
 + We need to record the output and grad_output of the feature maps to achieve Grad-CAM. In pytorch, the function `Hook` is defined for this purpose. Read the tutorial of [hook](https://pytorch.org/tutorials/beginner/former_torchies/nnft_tutorial.html#forward-and-backward-function-hooks) carefully.
 + More on [autograd](https://pytorch.org/tutorials/beginner/basics/autogradqs_tutorial.html) and [hooks](https://pytorch.org/tutorials/beginner/former_torchies/nnft_tutorial.html#forward-and-backward-function-hooks)
 + The pretrained model resnet34 doesn't have an activation function after its last layer, the output is indeed the `raw class scores`, you can use them directly. 
 + The size of feature maps is 7x7, so your heatmap will have the same size. You need to project the heatmap to the resized image (224x224, not the original one, before the normalization) to have a better observation. The function [`torch.nn.functional.interpolate`](https://pytorch.org/docs/stable/nn.functional.html?highlight=interpolate#torch.nn.functional.interpolate) may help.  
 + Here is the link of the paper [Grad-CAM: Visual Explanations from Deep Networks via Gradient-based Localization](https://arxiv.org/pdf/1610.02391.pdf)

Class: ‘pug, pug-dog’ | Class: ‘tabby, tabby cat’
- | - 
![alt](https://raw.githubusercontent.com/jacobgil/pytorch-grad-cam/master/examples/dog.jpg)| ![alt](https://raw.githubusercontent.com/jacobgil/pytorch-grad-cam/master/examples/cat.jpg)

In [ ]:
def Grad_CAM(input_tensor = dataset[index][0], model = resnet34, layer_name = 'layer4', sub_block = None):

    global activations
    activations = None
    global gradients
    gradients = None

    def feature_map_activations(self, input, output):
        #print('Inside ' + self.__class__.__name__ + ' forward')
        #print('output size:', output.data.size())
        global activations
        activations = output.data

    def feature_map_gradients(self, grad_input, grad_output):
        #print('Inside ' + self.__class__.__name__ + ' backward')
        #print('grad_output size:', grad_output[0].size())
        global gradients
        gradients = grad_output

    # Forward and backward hooks
    if not sub_block:
        activation_handle = model._modules[layer_name][-1].conv2.register_forward_hook(feature_map_activations)
        gradient_handle = model._modules[layer_name][-1].conv2.register_backward_hook(feature_map_gradients)
    else:
        activation_handle = model._modules[layer_name][sub_block].conv2.register_forward_hook(feature_map_activations)
        gradient_handle = model._modules[layer_name][sub_block].conv2.register_backward_hook(feature_map_gradients)
    
    # Forward pass
    input_image = input_tensor.view(1, 3, 224, 224)
    output = model(input_image)
    values, indices = torch.topk(output, 3)

    heatmaps = []

    for i in range(3):
        # Backward pass on the specific class
        if i > 0:
            output = model(input_image)
        model.zero_grad()
        output[0, indices[0][i]].backward()
        # class_interest = to

        # Importance weights
        num_feature_map = gradients[0].shape[1]
        size_feature_map = gradients[0].shape[2]
        alphas = torch.mean(gradients[0], dim=[2, 3]).view(1, num_feature_map, 1, 1)
        # Weighted combination of activation maps
        Lc = F.relu(torch.sum(torch.mul(activations, alphas), dim=1))
        # Upsampling
        Lc = F.interpolate(Lc.view(1, 1, size_feature_map, size_feature_map), size=(224, 224), mode='bilinear').view(1, 224, 224)
        heatmaps.append(Lc)

    # Remove hooks
    model._modules[layer_name]._forward_hooks.clear()
    model._modules[layer_name]._backward_hooks.clear()

    return heatmaps, indices, values

In [ ]:
from textwrap import wrap

def visualisations(input_image, heatmaps, indices, classes = classes):
    # Resizing
    input_image = Image.open(input_image).convert('RGB')
    input_image = input_image.resize((224, 224))
    # Plot
    fig, axs = plt.subplots(1, 4, figsize=(15, 15))
    for ax in axs:
        ax.axis('off')
    axs[0].imshow(input_image)
    axs[0].set_title('Input image')
    labels = [classes[x] for x in indices[0].numpy()]
    for i in range(3):
        axs[i+1].imshow(input_image)
        axs[i+1].imshow(heatmaps[i][0].detach().numpy(), alpha=0.5, cmap='jet')
        # axs[i+1].set_title(labels[i], wrap=True, fontsize=10)
        axs[i+1].set_title("\n".join(wrap(labels[i], 30)))

    plt.show()

In [ ]:
resnet34 = models.resnet34(weights='ResNet34_Weights.IMAGENET1K_V1')  # New PyTorch interface for loading weights!
resnet34.eval() # set the model to evaluation mode

# Your code here
for index in range(len(dataset)):
    input_image = dataset.imgs[index][0]
    input_tensor = dataset[index][0]
    heatmaps, indices, values = Grad_CAM(input_tensor, resnet34, 'layer4', sub_block= 2)
    print("Image {}".format(index + 1))
    print("Confidence scores:", values[0].detach().numpy())
    visualisations(input_image, heatmaps, indices)

#### **Heatmaps comparison**

We can make several comments on the heatmaps obtained for each class prediction.

The first one is that in most of the cases, the heatmaps are quite similar to what a human eye would focus on: the main object is well highlighted. This explains why the heatmaps are very often similar between the top-3 classes (Images 1, 2, 5, 6, 10, 12, 15, 18, 20). These heatmaps are quite rational: details such as tusks in Image 1 or antlers in Image 10 are highlighted by the model, thus proving that the model is focusing on the right features to make its predictions.

The second one is that we are using a high-level model, ResNet-34, yielding very good results in terms of image classification and able to discriminate between very similar classes. For example, in Images 3 or 4, the model focuses on different parts of the animal for the top-3 classes. This is interesting as it shows that the model is able to discriminate between similar classes by focusing on different parts of the image, and corroborate a remark made in the paper where Guided Grad-CAM highlighted stripes on the body of a tiger cat to distinguish it from other potential cat variety.

The third remark is that, corroborating once again a remark of the paper, *"seemingly unreasonable predictions have reasonable explanations"*. For example, in Image 16, it is hard to understand why the model would predict a cowboy boot instead of a sea lion: this prediction can however becomes less surprising when looking at the heatmap, which highlights solely the body of the animal whose skin could be confused with a leather-composed surface.

## Complementary questions:

### Try GradCAM on others convolutional layers, describe and comment the results

In the paper, the authors make the assumption that the last convolutional layers present the best compromise between "high-level semantics and detailed spatial information". This assumption was then sustained by empirical observations: "We find that Grad-CAM maps become progressively worse as we move to earlier convolutional layers as they have smaller receptive fields and only focus on less semantic local features".

This remark is consistent with what we have observed in our experiments. Indeed, the heatmaps obtained from the last convolutional layers are more focused on the object of interest, while the heatmaps obtained from the previous layers are more noisy and less interpretable. We can also make the remark that the heatmaps get more diffused as we move to the first layers, which is consistent with the fact that the receptive field of the neurons is smaller in the first layers, with an impression of edge detection rather than object detection on the first convolutional layer.

#### **Grad-CAM on penultimate feature map from Layer 4**

In [ ]:
resnet34 = models.resnet34(weights='ResNet34_Weights.IMAGENET1K_V1')  # New PyTorch interface for loading weights!
resnet34.eval() # set the model to evaluation mode

# Your code here
for index in range(5):
    input_image = dataset.imgs[index][0]
    input_tensor = dataset[index][0]
    heatmaps, indices, _ = Grad_CAM(input_tensor, resnet34, 'layer4', sub_block= 1)
    visualisations(input_image, heatmaps, indices)

#### **Grad-CAM on last feature map from Layer 3**

In [ ]:
resnet34 = models.resnet34(weights='ResNet34_Weights.IMAGENET1K_V1')  # New PyTorch interface for loading weights!
resnet34.eval() # set the model to evaluation mode

# Your code here
for index in range(5):
    input_image = dataset.imgs[index][0]
    input_tensor = dataset[index][0]
    heatmaps, indices, _ = Grad_CAM(input_tensor, resnet34, 'layer3')
    visualisations(input_image, heatmaps, indices)

#### **Grad-CAM on last feature map from Layer 2**

In [ ]:
resnet34 = models.resnet34(weights='ResNet34_Weights.IMAGENET1K_V1')  # New PyTorch interface for loading weights!
resnet34.eval() # set the model to evaluation mode

for index in range(5):
    input_image = dataset.imgs[index][0]
    input_tensor = dataset[index][0]
    heatmaps, indices, _ = Grad_CAM(input_tensor, resnet34, 'layer2')
    visualisations(input_image, heatmaps, indices)

#### **Grad-CAM on last feature map from Layer 1**

In [ ]:
resnet34 = models.resnet34(weights='ResNet34_Weights.IMAGENET1K_V1')  # New PyTorch interface for loading weights!
resnet34.eval() # set the model to evaluation mode

for index in range(5):
    input_image = dataset.imgs[index][0]
    input_tensor = dataset[index][0]
    heatmaps, indices, _ = Grad_CAM(input_tensor, resnet34, 'layer1')
    visualisations(input_image, heatmaps, indices)

### What are the principal contributions of GradCAM (the answer is in the paper) ?

The article "Grad-CAM: Visual Explanations from Deep Networks via Gradient-based Localization" presents several contributions. <br>

The first contribution, and perhaps the most important one, is that unlike previous visualizations methods such as Class Activation Mapping (CAM), **the Grad-CAM method can be applied to any CNN-based architecture without architectural changes or retraining**. To do so, it uses gradient signals on the target of interest (e.g., a class in image classification) flowing into the final convolutional layer to assign importance to each neuron of the layer in the decision-making process. Using these importance weights, it can then generate a coarse localization map, highlighting important regions in the image for predicting the concept. To quote the authors: "This allows our approach to be applied to off-the-shelf CNN-based architectures, including those for image captioning and visual question answering." They also show that this approach outperforms baselines when evaluated on localization and faithfulness to model.

The second contribution the authors made was to show that **their approach could be coupled with existing pixel-space gradient visualizations** (such as Guided Backpropagation or Deconvolution): they called this approach Guided Grad-CAM. This provides visualisations that are both high-resolution and class-discriminative, highlighting fine-grained details of importance while retaining class-discriminative features.

The third contribution the authors made was to show that using Grad-CAM **allows to perform diagnosis on model errors or to identify dataset bias**. The visualizations can indeed be used to explain model predictions that could seem irrationals at first sight, highlighting that *"seemingly unreasonable predictions have reasonable explanations"*. They also show that the visualizations can be used to identify dataset bias, by highlighting that the model is relying on features that are not relevant to the task at hand: the example they took was that a model trained to discriminate between "nurse" and "doctor" on a biased dataset used the person's face/hairstyle to make predictions, while an unbiased dataset allowed the model to rely on the person's clothing (short sleeves for nurses).

Another contribution was to use Grad-CAM to identify important neurons, and use this information combined with neurons' naming to **generate textual explanations for model decisions**. 

To conclude, the authors offers with Grad-CAM a method that is both architecture-agnostic and that can be used to generate high-resolution, class-discriminative visualizations. This method outperforms existing approaches both on interpretability and faithfulness to original model, and can be used to explain model predictions, to better expose the trustworthiness of a classifier, to identify dataset bias or to perform diagnosis on model errors.